In [1]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [16]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00


In [2]:
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoConfig,
    PreTrainedTokenizer,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
import torch
import os
import copy
from torch.optim import AdamW
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

In [3]:
import pandas as pd
import datasets
from typing import Dict, Sequence
import tqdm

In [4]:
def fmt_prompt(prompt):
    return f"### Instructions:\n Can you please translate this phrase or word to french? \n {prompt}\n\n### Response:\n Yes of course! Here is a french translation of that phrase: \n"

In [5]:
def _tokenize(
        strings: Sequence[str],
        tokenizer: PreTrainedTokenizer
) -> Dict:
    """tokenize examples"""
    tokenized_strings = [
        tokenizer(
            example,
            return_tensors='pt',
            padding=False,
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for example in strings
    ]

    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_strings]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item()
        for tokenized in tokenized_strings
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [6]:
def preprocess(
        samples: Sequence[str],
        tokenizer: PreTrainedTokenizer
    ) -> Dict:
    """Preprocess data for training by tokenizing"""
    sources = [f"{fmt_prompt(sources)}" for sources in samples["input"]]
    targets = [f"{translation}{tokenizer.eos_token}" for translation in samples["output"]]
    complete_examples = [s + t for s,t in zip(sources, targets)] # source + target -> "Can you translate this phrase for me? <|phrase|>, Sure thing, here is the french translation <|target|>"
    examples_tokenized, sources_tokenized = [
        _tokenize(strings, tokenizer) for strings in (complete_examples, sources)
    ]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_length in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_length] = -100 # Pytorch will ignore -100 during learning in c.e.l.
    return dict(input_ids=input_ids, labels=labels)


In [7]:
class MyDataSet(Dataset):
    """Dataset for fine-tuning model"""

    def __init__(self, tokenizer: PreTrainedTokenizer, paths: str, limit: int):
        super(MyDataSet, self).__init__()
        dataset = (
            datasets.load_dataset(
            "json",
            data_files=paths,
            split=f"train[0:{limit}]" if limit else "train",
            )
            .map(
                lambda samples: preprocess(samples, tokenizer),
                batched=True,
                batch_size=300,
            )
        )

        self.tokenizer = tokenizer
        self.input_ids = dataset["input_ids"]
        self.labels = dataset["labels"]
        self.size = len(dataset)

    def __len__(self) -> int:
        return len(self.input_ids)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        return dict(
            input_ids = torch.tensor(self.input_ids[idx]),
            labels = torch.tensor(self.labels[idx])
        )


In [8]:
from dataclasses import dataclass

@dataclass
class MyDataCollator(object):

    tokenizer: PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple(
            [instance[key] for instance in instances] for key in ("input_ids", "labels")
        )
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(
            labels, batch_first=True, padding_value=-100
        )

        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [9]:
path = "drive/MyDrive/phi-2/en-2-fr-translation.jsonl/"
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path = "./en-2-fr-translation.jsonl"

In [10]:
#### put everything together ######
tokenizer = AutoTokenizer.from_pretrained(
        'microsoft/phi-2',
        model_max_length=2048,
        padding_side="right",
        use_fast=False,
        pad_token="<|pad|>",
        trust_remote_code=True,
    )

dataset = MyDataSet(tokenizer, [path], 3000)

special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = "<|pad|>"
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = "<|eos|>"
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = "<|unk|>"

tokenizer.add_special_tokens(special_tokens_dict)
collator = MyDataCollator(tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [11]:
dataset.size

3000

In [12]:
# train and val split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size], generator)
print(f'train and validation dataset sizes: {len(train_dataset), len(val_dataset)}\n')

train and validation dataset sizes: (2400, 600)



In [13]:
# dataloader
batch_size = 4
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, collate_fn=collator)
val_dataloader = DataLoader(val_dataset, batch_size, collate_fn=collator)

In [14]:
#Quant Weights config
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    # store weights in 4bit
    bnb_4bit_quant_type = "nf4",
    # compute with 16 bit
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False
)

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2"]
)



In [17]:
# init model
model_name = "microsoft/phi-2"
new_model = "phi-2-QLoRA"

# config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype="auto",
    trust_remote_code=True,
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    low_cpu_mem_usage=True,
    device_map={"":0},
    revision="refs/pr/23"
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
#prepare_model_for_kbit_training---> This function basically helps to built the best model possible
model = prepare_model_for_kbit_training(model,use_gradient_checkpointing=True)

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [18]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50296, 2560)

In [19]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_params = sum([p.numel() for p in model.parameters()])
print(f"model params: {num_params}\n")

model params: 1516763256



In [20]:
from transformers import get_scheduler
import uuid

epochs = 1
train_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name='cosine', optimizer=optimizer, num_warmup_steps=0, num_training_steps=train_steps)

In [21]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
model.to(device)

ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [26]:
def should_run_eval(total_steps, freq, current_step):
    return current_step % (total_steps // freq) == 0

def eval(model, val_data, wandb):
    print("evaluating model...\n")
    model.eval()
    losses = 0.0
    for step, batch in enumerate(val_data):
        batch = {
            "input_ids": batch["input_ids"].to(model.device),
            "labels": batch["labels"].to(model.device),
            "attention_mask": batch["attention_mask"].to(model.device),
        }
        with torch.no_grad():
            outputs = model(**batch)

        # record loss
        loss = outputs.loss
        losses += loss.float()
    val_loss = losses / (step + 1)
    wandb.log(
        {
            "val_loss": val_loss
        }
    )

    return val_loss

def save_model(model, outpath: str, current_epoch: int, current_step: int):
    print(f"saving model at epoch: {current_epoch}, step: {current_step}")
    outpath += f"/model"
    model.save_pretrained(outpath)

def run_stats(pbar, wandb, epoch, step, loss):

    wandb.log({
        "current_loss": loss,
        "current_epoch": epoch
    })

    current_loss = f"{loss:.4f}"

    pbar.set_description(f"Epoch {epoch} :: Step {step} :: Loss {current_loss}")

In [27]:
# from model import train_model
from tqdm.auto import tqdm

def train_model(model, epochs, train_dataloader, val_dataloader, train_steps, optimizer, lr_scheduler, wandb):
    pbar = tqdm(range(train_steps))

    run_id = str(uuid.uuid4())
    print(f"model id :: {run_id}")
    output_dir = f"drive/MyDrive/phi-2/outputs/{run_id}"
    model.train()
    for epoch in range(epochs):
        current_epoch = epoch + 1
        for step, batch in enumerate(train_dataloader):
            current_step = step + 1

            batch = {
                "input_ids": batch["input_ids"].to(model.device),
                "labels": batch["labels"].to(model.device),
                "attention_mask": batch["attention_mask"].to(model.device),
            }

            # forward
            outputs = model(**batch)
            loss = outputs.loss

            # backward
            loss.backward()

            # log results
            loss = loss.detach()
            run_stats(pbar, wandb, epoch, step, loss.item())

            # update weights
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            # evaluate and save model
            if should_run_eval(len(train_dataloader), 5, current_step):
                val_loss = eval(model, val_dataloader, wandb)

                save_model(model, output_dir, current_epoch, current_step)
                model.train()
            pbar.update(1)

In [24]:
!pip install -U wandb
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [25]:
wandb.init(
    # set the wandb project where this run will be logged
    project="phi-2-fine-tune",

    # track hyperparameters and run metadata
    config={
        "model_name": model_name,
        "run_id": "test_run",
        "epochs": 1,
        "batch_size": batch_size
    }
)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [28]:
print("beginning model training...\n")
train_model(model, epochs, train_dataloader, val_dataloader, train_steps, optimizer, lr_scheduler, wandb)

print("complete")

beginning model training...



  0%|          | 0/600 [00:00<?, ?it/s]

model id :: 26cb20ee-bfa0-4a76-a8e4-9787601000b4


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
# Empty VRAM
del model
import gc
gc.collect()

90